In this case study, I am using Juptyer Notebook and SQLite to collect certain information from a database containing information about a country club's facilities and their bookings by members and guests of the club. For this case study I will connect to a local instance of the database using Python and SQLite. 

# Imports

In [37]:
import sqlite3
from sqlite3 import Error
import pandas as pd

# Connecting to the database

Create a function to create the connection to a local instance of the database using SQLite.

In [50]:
def create_connection(db_file):
    """ create a database connection to the SQLite database
        specified by the db_file
    :param db_file: database file
    :return: Connection object or None
    """
    conn = None
    try:
        conn = sqlite3.connect(db_file)
        #print(sqlite3.version)
    except Error as e:
        print(e)
 
    return conn

 
def select_all_tasks(conn, query1):
    """
    Query all rows in the tasks table
    :param conn: the Connection object
    :return:
    """
    cur = conn.cursor()
    
    cur.execute(query1)
 
    rows = cur.fetchall()

#    for row in rows:
 #       print(row)
    return rows

# SQL queries

Some of the facilities charge a fee to members, but some do not. The following SQL query produces a list of the names of the facilities that do.

In [51]:
query1 = """
SELECT name
FROM `Facilities`
WHERE membercost !=0
AND membercost IS NOT NULL
        """

# create a database connection
database = "sqlite_db_pythonsqlite.db"
conn = create_connection(database)
with conn: 
    result = select_all_tasks(conn, query1)

table = pd.DataFrame(result, columns = ['facility'])
table

,facility
0,Tennis Court 1
1,Tennis Court 2
2,Massage Room 1
3,Massage Room 2
4,Squash Court


How many facilities do not charge a fee to members?

In [52]:
query1 = """
SELECT COUNT( facid )
FROM `Facilities`
WHERE membercost =0
AND membercost IS NOT NULL
        """

# create a database connection
database = "sqlite_db_pythonsqlite.db"
conn = create_connection(database)
with conn: 
    result = select_all_tasks(conn, query1)

result[0][0]

4

The following SQL query shows a list of facilities that charge a fee to members, where the fee is less than 20% of the facility's monthly maintenance cost.

In [53]:
query1 = """
SELECT facid, name, membercost, monthlymaintenance
FROM `Facilities`
WHERE membercost >0
AND membercost IS NOT NULL
AND membercost < 0.2 * monthlymaintenance
        """

# create a database connection
database = "sqlite_db_pythonsqlite.db"
conn = create_connection(database)
with conn: 
    result = select_all_tasks(conn, query1)

table = pd.DataFrame(result, columns = ['facid','facility', 'membercost', 'monthlymaintenance'])
table

,facid,facility,membercost,monthlymaintenance
0,0,Tennis Court 1,5.0,200
1,1,Tennis Court 2,5.0,200
2,4,Massage Room 1,9.9,3000
3,5,Massage Room 2,9.9,3000
4,6,Squash Court,3.5,80


The following SQL query retrieves the details of facilities with IDs 1 and 5.

In [56]:
query1 = """
SELECT *
FROM `Facilities`
WHERE facid
IN ( 1, 5 )
        """

# create a database connection
database = "sqlite_db_pythonsqlite.db"
conn = create_connection(database)
with conn: 
    result = select_all_tasks(conn, query1)

table = pd.DataFrame(result,columns=['facid','facility','membercost','guestcost','initialoutlay','monthlymaintenance'])
table

,facid,facility,membercost,guestcost,initialoutlay,monthlymaintenance
0,1,Tennis Court 2,5.0,25,8000,200
1,5,Massage Room 2,9.9,80,4000,3000


The following query produces a list of facilities, with each labelled as 'cheap' or 'expensive', depending on if their monthly maintenance cost is more than $100.

In [58]:
query1 = """
SELECT name, monthlymaintenance,
CASE
WHEN monthlymaintenance >100
THEN 'expensive'
ELSE 'cheap'
END AS label
FROM `Facilities`
        """

# create a database connection
database = "sqlite_db_pythonsqlite.db"
conn = create_connection(database)
with conn: 
    result = select_all_tasks(conn, query1)

table = pd.DataFrame(result, columns = ['facility', 'monthlymaintenance', 'label'])
table

,facility,monthlymaintenance,label
0,Tennis Court 1,200,expensive
1,Tennis Court 2,200,expensive
2,Badminton Court,50,cheap
3,Table Tennis,10,cheap
4,Massage Room 1,3000,expensive
5,Massage Room 2,3000,expensive
6,Squash Court,80,cheap
7,Snooker Table,15,cheap
8,Pool Table,15,cheap


The following query collects the first and last name of the last member who signed up.

In [60]:
query1 = """
SELECT firstname, surname
FROM `Members`
WHERE DATE( joindate ) = (
SELECT MAX( DATE( joindate ) )
FROM `Members` )
        """

# create a database connection
database = "sqlite_db_pythonsqlite.db"
conn = create_connection(database)
with conn: 
    result = select_all_tasks(conn, query1)

table = pd.DataFrame(result, columns = ['firstname', 'surname'])
table

,firstname,surname
0,Darren,Smith


The following query produces a list of all members who have used a tennis court.

In [62]:
query1 = """
SELECT m.firstname, m.surname, f.name
FROM `Bookings` AS b
INNER JOIN `Members` AS m ON b.memid = m.memid
INNER JOIN `Facilities` AS f ON b.facid = f.facid
WHERE name LIKE 'Tennis%'
AND m.surname != 'GUEST'
AND m.firstname != 'GUEST'
GROUP BY m.surname, m.firstname, f.name
ORDER BY m.surname
        """

# create a database connection
database = "sqlite_db_pythonsqlite.db"
conn = create_connection(database)
with conn: 
    result = select_all_tasks(conn, query1)

table = pd.DataFrame(result, columns = ['firstname', 'surname','facility'])
table

,firstname,surname,facility
0,Florence,Bader,Tennis Court 1
1,Florence,Bader,Tennis Court 2
2,Anne,Baker,Tennis Court 1
3,Anne,Baker,Tennis Court 2
4,Timothy,Baker,Tennis Court 1
5,Timothy,Baker,Tennis Court 2
6,Tim,Boothe,Tennis Court 1
7,Tim,Boothe,Tennis Court 2
8,Gerald,Butters,Tennis Court 1
9,Gerald,Butters,Tennis Court 2


The following query produces a list of bookings on the day of 2012-09-14 which will cost the member (or guest) more than $30. Here, guests have different costs to members (the listed costs are per half-hour 'slot'), and the guest user's ID is always 0. This is ordered by descending cost, and no subqueries are used yet. The query is performed with joins. 

In [75]:
query1 = """
SELECT f.name, m.firstname || ' ' || m.surname AS user_name,
       (CASE WHEN m.firstname = 'GUEST' THEN b.slots * f.guestcost
             WHEN m.firstname != 'GUEST' THEN b.slots * f.membercost
        END)
        AS total_cost
FROM `Bookings` AS b
INNER JOIN `Members` AS m ON b.memid = m.memid
INNER JOIN `Facilities` AS f ON b.facid = f.facid
WHERE b.starttime LIKE '2012-09-14%'
AND total_cost > 30
"""
# create a database connection
database = "sqlite_db_pythonsqlite.db"
conn = create_connection(database)
with conn: 
    result = select_all_tasks(conn, query1)

table = pd.DataFrame(result, columns = ['facility','user_name', 'total_cost'])
table

,facility,user_name,total_cost
0,Tennis Court 1,GUEST GUEST,75.0
1,Tennis Court 1,GUEST GUEST,75.0
2,Tennis Court 2,GUEST GUEST,75.0
3,Tennis Court 2,GUEST GUEST,150.0
4,Massage Room 1,GUEST GUEST,160.0
5,Massage Room 1,GUEST GUEST,160.0
6,Massage Room 1,Jemima Farrell,39.6
7,Massage Room 1,GUEST GUEST,160.0
8,Massage Room 2,GUEST GUEST,320.0
9,Squash Court,GUEST GUEST,70.0


This time, I will produce the same result as above but using a subquery in the FROM clause

In [74]:
query1 = """
SELECT name, user_name,
       (CASE WHEN user_name = 'GUEST GUEST' THEN slots * guestcost
             WHEN user_name != 'GUEST GUEST' THEN slots * membercost
        END)
        AS total_cost
FROM 
(SELECT f.name, m.firstname || ' ' || m.surname as user_name, b.slots, f.guestcost, f.membercost
 FROM `Bookings` AS b
    INNER JOIN `Members` AS m ON b.memid = m.memid
    INNER JOIN `Facilities` AS f ON b.facid = f.facid
    WHERE b.starttime LIKE '2012-09-14%') AS subquery
WHERE total_cost > 30
"""
# create a database connection
database = "sqlite_db_pythonsqlite.db"
conn = create_connection(database)
with conn: 
    result = select_all_tasks(conn, query1)

table = pd.DataFrame(result, columns = ['facility','user_name', 'total_cost'])
table

,facility,user_name,total_cost
0,Tennis Court 1,GUEST GUEST,75.0
1,Tennis Court 1,GUEST GUEST,75.0
2,Tennis Court 2,GUEST GUEST,75.0
3,Tennis Court 2,GUEST GUEST,150.0
4,Massage Room 1,GUEST GUEST,160.0
5,Massage Room 1,GUEST GUEST,160.0
6,Massage Room 1,Jemima Farrell,39.6
7,Massage Room 1,GUEST GUEST,160.0
8,Massage Room 2,GUEST GUEST,320.0
9,Squash Court,GUEST GUEST,70.0


The next query produces a list of facilities with a total revenue less than 1000. 

In [77]:
query1 = """
SELECT name, revenue
FROM
(SELECT name, SUM(total_cost) AS revenue
FROM
(SELECT f.name, f.name, m.firstname || ' ' || m.surname as user_name, b.starttime,
       (CASE WHEN m.firstname = 'GUEST' THEN b.slots * f.guestcost
             WHEN m.firstname != 'GUEST' THEN b.slots * f.membercost
        END)
        AS total_cost
FROM `Bookings` AS b
INNER JOIN `Members` AS m ON b.memid = m.memid
INNER JOIN `Facilities` AS f ON b.facid = f.facid) AS subquery
GROUP BY name
ORDER BY revenue DESC) AS nested_subquery
WHERE revenue < 1000
"""
# create a database connection
database = "sqlite_db_pythonsqlite.db"
conn = create_connection(database)
with conn: 
    result = select_all_tasks(conn, query1)

table = pd.DataFrame(result, columns = ['facility','revenue'])
table

,facility,revenue
0,Pool Table,270
1,Snooker Table,240
2,Table Tennis,180


The next query produces a report of members and who recommended them in alphabetic surname,firstname order 

In [80]:
query1 = """
SELECT m1.surname, m1.firstname, m2.surname || ' ' || m2.firstname AS recommended_by
FROM Members as m1
INNER JOIN Members as m2
ON m1.recommendedby = m2.memid
WHERE m1.recommendedby != 0
ORDER BY m1.surname, m1.firstname
        """

# create a database connection
database = "sqlite_db_pythonsqlite.db"
conn = create_connection(database)
with conn: 
    result = select_all_tasks(conn, query1)

table = pd.DataFrame(result, columns = ['surname', 'firstname', 'recommendedby'])
table

,surname,firstname,recommendedby
0,Bader,Florence,Stibbons Ponder
1,Baker,Anne,Stibbons Ponder
2,Baker,Timothy,Farrell Jemima
3,Boothe,Tim,Rownam Tim
4,Butters,Gerald,Smith Darren
5,Coplin,Joan,Baker Timothy
6,Crumpet,Erica,Smith Tracy
7,Dare,Nancy,Joplette Janice
8,Genting,Matthew,Butters Gerald
9,Hunt,John,Purview Millicent


Facilities with their usage by member, but not guests

In [83]:
query1 = """
SELECT f.name, m.firstname || ' ' || m.surname as user_name, SUM(slots) as facility_usage 
FROM Bookings as b 
INNER JOIN Facilities as f ON b.facid = f.facid 
INNER JOIN Members as m ON b.memid = m.memid 
WHERE m.firstname != 'GUEST'
GROUP BY f.name, user_name 
ORDER BY 1, 3 DESC
        """

# create a database connection
database = "sqlite_db_pythonsqlite.db"
conn = create_connection(database)
with conn: 
    result = select_all_tasks(conn, query1)

table = pd.DataFrame(result, columns = ['surname', 'firstname', 'recommendedby'])
table

,surname,firstname,recommendedby
0,Badminton Court,Darren Smith,432
1,Badminton Court,Tracy Smith,102
2,Badminton Court,Anna Mackenzie,96
3,Badminton Court,Gerald Butters,63
4,Badminton Court,Ponder Stibbons,48
...,...,...,...
197,Tennis Court 2,Millicent Purview,3
198,Tennis Court 2,Jemima Farrell,3
199,Tennis Court 2,Jack Smith,3
200,Tennis Court 2,Henrietta Rumney,3


In [84]:
query1 = """
SELECT f.name,
(CASE WHEN starttime LIKE '2012-07%' THEN 'July' 
 WHEN starttime LIKE '2012-08%' THEN 'August' 
 WHEN starttime LIKE '2012-09%' THEN 'September' 
 END) AS month,
 SUM(b.slots) AS total_usage
 FROM Bookings as b
 INNER JOIN Facilities AS f
 ON b.facid = f.facid
GROUP BY month, name
ORDER BY 2, 3 DESC
        """

# create a database connection
database = "sqlite_db_pythonsqlite.db"
conn = create_connection(database)
with conn: 
    result = select_all_tasks(conn, query1)

table = pd.DataFrame(result, columns = ['facility', 'month', 'total_usage'])
table

,facility,month,total_usage
0,Massage Room 1,August,492
1,Tennis Court 2,August,483
2,Tennis Court 1,August,459
3,Badminton Court,August,459
4,Squash Court,August,400
5,Snooker Table,August,326
6,Pool Table,August,322
7,Table Tennis,August,304
8,Massage Room 2,August,82
9,Tennis Court 1,July,270
